# Transcribe any audio stream with Deepgram!

**Make a copy of this notebook into your own drive, and follow the instructions below!** 🥳🥳🥳


----------------------------

# Get started:

1) Copy this notebook (`File > Save a copy in Drive`) or download the .ipynb (`File > Download > Download as .ipynb`). 

2) Follow the instructions below!

----------------------------
# Instructions:
Running the following cells will allow you to transcribe any audio stream you wish. The comments below point out the variables you can manipulate to modify your output.

Also, this notebook comes with a video tutorial! Here: https://youtu.be/dq4AiiiaAsY

And by the way, if you haven't yet signed up for Deepgram, check out this link: https://dpgr.am/streaming-notebook

# Step 1: Dependencies

Run this cell to download all necessary dependencies.

Note: You can run a cell by clicking the play button on the left or by clicking on the cell and pressing `shift`+`ENTER` at the same time. (Or `shift` + `return` on Mac).

# Step 2: Live Transcription

Run this cell to livestream audio from any source! By default, we are streaming from BBC radio:  'http://stream.live.vc.bbcmedia.co.uk/bbc_radio_fourlw_online_nonuk'

Set the following variables as necessary:


*   `DEEPGRAM_API_KEY` = Your personal Deepgram API key
*   `URL` = The URL you wish to stream from. Again, we're streaming from BBC Radio by default, but if you wish to stream from another source just replace this URL.
*  `PARAMS` = The starter code here should give you a well-punctuated English transcript. However, if you wish to change the language, use a different model, or modify the output, check out our docs to figure out the best parameter configuration for you: https://developers.deepgram.com/documentation/
*  `TIME_LIMIT` = The number of seconds you wish to transcribe the livestream.Set this equal to `float(inf)` if you wish to stream forever. (Or at least until you wish to cut off the function manually.)
*  `TRANSCRIPT_ONLY` = Set this variable to `True` if you wish only to see the words you're transcribing. Set it to `False` if you wish to see the full JSON responses (which include word-level timestamps, metadata, and confidence levels).

To run the cell, click the play button or press `Shift`+`Enter` on your keyboard. (Or `Shift`+`return` if you're on Mac.)

-------

Note: if you're already a Deepgram user, and you're getting an error in this cell the most common issues are:

1. You may need to update your installation of `deepgram-sdk`.
2. You may need to check how many credits you have left in your Deepgram account.



In [ ]:
from deepgram import Deepgram
import asyncio
import aiohttp
import time

# Your personal API key
DEEPGRAM_API_KEY = '🔑🔑🔑 Your API Key here! 🔑🔑🔑'

# URL for the real-time streaming audio you would like to transcribe
URL = 'http://stream.live.vc.bbcmedia.co.uk/bbc_radio_fourlw_online_nonuk'

# Fill in these parameters to adjust the output as you wish!
# See our docs for more info: https://developers.deepgram.com/documentation/ 
PARAMS = {"punctuate": True, 
          "numerals": True,
          "model": "general", 
          "language": "en-US",
          "tier": "enhanced" }

# The number of *seconds* you wish to transcribe the livestream.
# Set this equal to `float(inf)` if you wish to stream forever.
# (Or at least until you wish to cut off the function manually.)
TIME_LIMIT = 30

# Set this variable to `True` if you wish only to 
# see the transcribed words, like closed captions. 
# Set it to `False` if you wish to see the raw JSON responses.
TRANSCRIPT_ONLY = True

'''
Function object.

Input: JSON data sent by a live transcription instance, which is named 
`deepgramLive` in main().

Output: The printed transcript obtained from the JSON object
'''
def print_transcript(json_data):
    try:
      print(json_data['channel']['alternatives'][0]['transcript'])
    except KeyError:
      print()

async def main():
    start = time.time()
    # Initializes the Deepgram SDK
    deepgram = Deepgram(DEEPGRAM_API_KEY)
    # Create a websocket connection to Deepgram
    try:
        deepgramLive = await deepgram.transcription.live(PARAMS)
    except Exception as e:
        print(f'Could not open socket: {e}')
        return

    # Listen for the connection to close
    deepgramLive.registerHandler(deepgramLive.event.CLOSE, 
                                 lambda _: print('✅ Transcription complete! Connection closed. ✅'))

    # Listen for any transcripts received from Deepgram & write them to the console
    if TRANSCRIPT_ONLY:
        deepgramLive.registerHandler(deepgramLive.event.TRANSCRIPT_RECEIVED, 
                                  print_transcript)
    else:
        deepgramLive.registerHandler(deepgramLive.event.TRANSCRIPT_RECEIVED, print)

    # Listen for the connection to open and send streaming audio from the URL to Deepgram
    async with aiohttp.ClientSession() as session:
        async with session.get(URL) as audio:
            while time.time() - start < TIME_LIMIT:
                data = await audio.content.readany()
                if data:
                    deepgramLive.send(data)
                else:
                    break

    # Indicate that we've finished sending data by sending the customary 
    # zero-byte message to the Deepgram streaming endpoint, and wait 
    # until we get back the final summary metadata object
    await deepgramLive.finish()

await main()